# <center> Movie Recommendation Engine and EDA

In [ ]:
import os
import plotly.express as px
import numpy as np
from datetime import datetime
import pandas as pd
import plotly.graph_objects as go
import seaborn as sns
import re
from sklearn.neighbors import NearestNeighbors
import random
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
!pip install mlxtend

In [ ]:
dataNames = pd.read_csv(r"C:\Users\Ajend\OneDrive\Documents\movies.csv")
dataRatings = pd.read_csv(r"C:\Users\Ajend\OneDrive\Documents\ratings.csv")

In [ ]:
dataRatings.head()  # First dataset 

In [ ]:
dataNames.head()  # Second dataset

In [ ]:
# Inner join of two datasets (common column is movieId)
data = pd.merge(dataRatings, dataNames, how='inner')
data.head()

In [ ]:
# Number of Users who rated at least one movie:
print("Number of Users who rated at least one move: ", data.userId.nunique())
print("-"*25)

# Number of Movies in the dataset:
print("Number of Movies in the dataset:", data.title.nunique())
print("-"*25)

# Unique of Rating points in the dataset:
print("Unique Rating points:", data.rating.unique())

# Creating new Features and Data Preprocessing


**In this step, I will create new features for more detailed EDA and then, I will prepare the dataset for further steps.**

In [ ]:
# Extracting movie release years into one column
data['movie_year'] = data.title.str.extract('.*\((.*)\).*')
data.head()

**From now on, we can find the movie years on the movie_year column**

In [ ]:
# Removing year from the movie titles
data['title'] = data.title.str.split('(').str[0].str[:-1]
data.head()

**timestamp column has not readable data points. We cannot use this values for our analysis. Hence, I will convert this values into Readable format.**

In [ ]:
def UNIX_to_Readable(df):
    return pd.to_datetime(datetime.fromtimestamp(df).strftime('%Y-%m-%d %H:%M:%S'))


# Converting Unix date-format to readable format
data.timestamp = data.timestamp.apply(UNIX_to_Readable)
data.head()

In [ ]:
# Removing decimal values to the ceiling value to decrease number of rating classes
data.rating = np.ceil(data.rating)
print("Unique Rating Points:", data.rating.unique())

In [ ]:
_ = data.title.value_counts()
fig = px.histogram(_, x=_, opacity=0.85, marginal='box',
                   labels={
                       'x': 'Number of Ratings taken'})
fig.update_traces(marker=dict(line=dict(color='#000000', width=1)))
fig.update_layout(title_text='Distribution of the Number of Ratings taken by the Movies',
                  title_x=0.5, title_font=dict(size=20))
fig.update_layout({'plot_bgcolor': 'rgba(0, 0, 0, 0)'})
fig.show()

In [ ]:
# Removing movies that rated less than 10
movieFrequency_greater_10 = data['movieId'].value_counts()[data['movieId'].value_counts() >= 10].index
data = data[data.movieId.isin(movieFrequency_greater_10)]

print("Minimum Number of Rated Movies after Drop:", data.title.value_counts().nsmallest(5))   # So we achieved to obtain number of ratings taken by users minimum 10

# Exploratory Data Analysis

# Frequency of the Ratings


In [ ]:
rating_val_count = data.rating.value_counts()
fig = px.bar(rating_val_count, x=rating_val_count.index, y=rating_val_count, text=rating_val_count,
             labels={
                 "index": "Ratings",
                 'y': 'Number of Ratings'},
             color=rating_val_count
             )
fig.update_traces(textposition='outside')
fig.update_layout(title_text='Frequency of the Ratings',
                  title_x=0.5, title_font=dict(size=24))
fig.update_traces(marker=dict(line=dict(color='#000000', width=2)))
fig.update_layout({'plot_bgcolor': 'rgba(0, 0, 0, 0)'})
fig.show()

**According to figure above, most of the users rated movies 4.**

# Top Frequent the Movie Genres

In [ ]:
genres_value_counts = data['genres'].str.split('|', expand=True).stack().value_counts()
fig = px.bar(genres_value_counts, x=genres_value_counts.index, y=genres_value_counts, text=genres_value_counts,
             labels={
                 "index": "Genres",
                 'y': 'Frequency'},
             color=genres_value_counts
             )
fig.update_traces(textposition='outside')
fig.update_layout(title_text='Top Frequent the Movie Genres',
                  title_x=0.5, title_font=dict(size=24))
fig.update_traces(marker=dict(line=dict(color='#000000', width=2)))
fig.update_layout({'plot_bgcolor': 'rgba(0, 0, 0, 0)'})
fig.show()

**Drama, Comedy and Action genres are the top frequent genres according to our dataset. These genres are rated most of the users.**

# Number of Movies for each Genre

In [ ]:
unique_movies = data.drop_duplicates('title')
unique_movies = unique_movies['genres'].str.split('|', expand=True).stack().value_counts()
fig = px.bar(unique_movies, x=unique_movies.index, y=unique_movies, text=unique_movies,
             labels={
                 "index": "Genres",
                 'y': 'Number of Movies'},
             color=unique_movies
             )
fig.update_traces(textposition='outside')
fig.update_layout(title_text='Number of Movies for each Genre',
                  title_x=0.5, title_font=dict(size=24))
fig.update_traces(marker=dict(line=dict(color='#000000', width=2)))
fig.update_layout({'plot_bgcolor': 'rgba(0, 0, 0, 0)'})
fig.show()

# Year Difference Between Release Date of the Movie and the Rating Date


In [ ]:
data['timeDifferenceAfterRelease'] = data.timestamp.dt.year - data.movie_year.astype(int)
fig = px.histogram(data, x='timeDifferenceAfterRelease', opacity=0.85, marginal='box',
                   labels={
                       'timeDifferenceAfterRelease': 'Time Difference After Release (years)'}
                   )
fig.update_traces(marker=dict(line=dict(color='#000000', width=1)))
fig.update_layout(title_text='Year Difference Between Release Date of the Movie and the Rating Date',
                  title_x=0.5, title_font=dict(size=20))
fig.update_layout({'plot_bgcolor': 'rgba(0, 0, 0, 0)'})
fig.show()

# Number of Ratings given by the Users Monthly


In [ ]:
num_of_rating_weekly = data.resample('m', on='timestamp').size()
fig = px.line(num_of_rating_weekly, x=num_of_rating_weekly.index, y=num_of_rating_weekly,
              labels={'y': 'Number of Ratings given',
                      'timestamp': 'Months'})
fig.update_layout(title_text='Number of Ratings given by the Users Monthly',
                  title_x=0.5, title_font=dict(size=20))
fig.update_layout({'plot_bgcolor': 'rgba(0, 0, 0, 0)'})
fig.update_traces(line=dict(width=3))
fig.show()

# Number of Movies Released each Year


In [ ]:
unique_title = data.drop_duplicates(['title'])  # Dropping duplicate movie titles
unique_title = unique_title.movie_year.value_counts().reset_index().sort_values('index')  # Count of each years movie frequency
fig = px.line(unique_title, x='index', y='movie_year',
              labels={
                  "index": "Movie Release Year",
                  'movie_year': 'Number of Movies Released'})
fig.update_layout(title_text='Number of Movies Released each Year',
                  title_x=0.5, title_font=dict(size=24))
fig.update_traces(marker=dict(line=dict(color='#000000', width=2)))
fig.update_layout({'plot_bgcolor': 'rgba(0, 0, 0, 0)'})
fig.show()

# Top 25 Years with the most Number of Movies Released


In [ ]:
unique_title = data.drop_duplicates(['title'])  # Dropping duplicate movie titles
unique_title = unique_title.movie_year.value_counts().head(25)  # Count of each years movie frequency
fig = px.bar(unique_title, x=unique_title.index, y=unique_title, text=unique_title,
             labels={
                 "index": "Movie Release Year",
                 'y': 'Number of Movies Released'},
             color=unique_title
             )
fig.update_traces(textposition='outside')
fig.update_layout(title_text='Top 25 Years with the most Number of Movies Released',
                  title_x=0.5, title_font=dict(size=24))
fig.update_traces(marker=dict(line=dict(color='#000000', width=2)))
fig.update_layout({'plot_bgcolor': 'rgba(0, 0, 0, 0)'})
fig.show()


In [ ]:
genre_vs_rating = data.groupby(['genres', 'rating']).size().unstack().fillna(0)
year_vs_rating = data.groupby(['movie_year', 'rating']).size().unstack().fillna(0)
movie_vs_rating = data.groupby(['title', 'rating']).size().unstack().fillna(0)

In [ ]:
# Let's calculate the Weighted Average for dataframe rows
def Weighted_Average(df):
    x = []
    for i in range(0, df.shape[0]):
        x.append((np.average(df.iloc[i].index, weights=df.iloc[i].values, axis=0)).round(2))
    return x

# Weighted Average calculation for each movie_vs_rating rows
movie_vs_rating['weightedAverage'] = Weighted_Average(movie_vs_rating)
movie_vs_rating.sort_values('weightedAverage', ascending=False).head()

# Weighted Average calculation for each year_vs_rating rows
year_vs_rating['weightedAverage'] = Weighted_Average(year_vs_rating)

# Weighted Average calculation for each genre_vs_rating rows
genre_vs_rating['weightedAverage'] = Weighted_Average(genre_vs_rating)

# Top 15 Movie with the highest Weighted Averages


In [ ]:
fig = px.bar(movie_vs_rating, x=movie_vs_rating['weightedAverage'].nlargest(15).index,
             y=movie_vs_rating['weightedAverage'].nlargest(15),
             text=movie_vs_rating['weightedAverage'].nlargest(15),
             labels={
                 "x": "Movies",
                 'y': 'Weighted Rating Averages'},
             color=movie_vs_rating['weightedAverage'].nlargest(15)
             )
fig.update_traces(textposition='outside')
fig.update_layout(title_text='Top 15 Movie with the highest Weighted Averages',
                  title_x=0.5, title_font=dict(size=24))
fig.update_traces(marker=dict(line=dict(color='#000000', width=2)))
fig.update_layout({'plot_bgcolor': 'rgba(0, 0, 0, 0)'})
fig.show()

# Top 15 Movie with the smallest Weighted Averages


In [ ]:
fig = px.bar(movie_vs_rating, x=movie_vs_rating['weightedAverage'].nsmallest(15).index,
             y=movie_vs_rating['weightedAverage'].nsmallest(15),
             text=movie_vs_rating['weightedAverage'].nsmallest(15),
             labels={
                 "x": "Movies",
                 'y': 'Weighted Rating Averages'},
             color=movie_vs_rating['weightedAverage'].nsmallest(15)
             )
fig.update_traces(textposition='outside')
fig.update_layout(title_text='Top 15 Movie with the smallest Weighted Averages',
                  title_x=0.5, title_font=dict(size=24))
fig.update_traces(marker=dict(line=dict(color='#000000', width=2)))
fig.update_layout({'plot_bgcolor': 'rgba(0, 0, 0, 0)'})
fig.show()

# Top 15 Years with the highest Weighted Averages


In [ ]:
fig = px.bar(year_vs_rating, x=year_vs_rating['weightedAverage'].nlargest(15).index,
             y=year_vs_rating['weightedAverage'].nlargest(15),
             text=year_vs_rating['weightedAverage'].nlargest(15),
             labels={
                 "x": "Years",
                 'y': 'Weighted Rating Averages'},
             color=year_vs_rating['weightedAverage'].nlargest(15)
             )
fig.update_traces(textposition='outside')
fig.update_layout(title_text='Top 15 Years with the highest Weighted Averages',
                  title_x=0.5, title_font=dict(size=24))
fig.update_traces(marker=dict(line=dict(color='#000000', width=2)))
fig.update_layout({'plot_bgcolor': 'rgba(0, 0, 0, 0)'})
fig.show()

# Top 15 Years with the smallest Weighted Averages


In [ ]:
fig = px.bar(year_vs_rating, x=year_vs_rating['weightedAverage'].nsmallest(15).index,
             y=year_vs_rating['weightedAverage'].nsmallest(15),
             text=year_vs_rating['weightedAverage'].nsmallest(15),
             labels={
                 "x": "Years",
                 'y': 'Weighted Rating Averages'},
             color=year_vs_rating['weightedAverage'].nsmallest(15)
             )
fig.update_traces(textposition='outside')
fig.update_layout(title_text='Top 15 Years with the smallest Weighted Averages',
                  title_x=0.5, title_font=dict(size=24))
fig.update_traces(marker=dict(line=dict(color='#000000', width=2)))
fig.update_layout({'plot_bgcolor': 'rgba(0, 0, 0, 0)'})
fig.show()

# Top 15 Genres with the highest Weighted Averages


In [ ]:
fig = px.bar(genre_vs_rating, x=genre_vs_rating['weightedAverage'].nlargest(15).index,
             y=genre_vs_rating['weightedAverage'].nlargest(15),
             text=genre_vs_rating['weightedAverage'].nlargest(15),
             labels={
                 "x": "Genres",
                 'y': 'Weighted Rating Averages'},
             color=genre_vs_rating['weightedAverage'].nlargest(15)
             )
fig.update_traces(textposition='outside')
fig.update_layout(title_text='Top 15 Genres with the highest Weighted Averages',
                  title_x=0.5, title_font=dict(size=24))
fig.update_traces(marker=dict(line=dict(color='#000000', width=2)))
fig.update_layout({'plot_bgcolor': 'rgba(0, 0, 0, 0)'})
fig.show()

# Top 15 Genres with the smallest Weighted Averages


In [ ]:
fig = px.bar(genre_vs_rating, x=genre_vs_rating['weightedAverage'].nsmallest(15).index,
             y=genre_vs_rating['weightedAverage'].nsmallest(15),
             text=genre_vs_rating['weightedAverage'].nsmallest(15),
             labels={
                 "x": "Genres",
                 'y': 'Weighted Rating Averages'},
             color=genre_vs_rating['weightedAverage'].nsmallest(15)
             )
fig.update_traces(textposition='outside')
fig.update_layout(title_text='Top 15 Genres with the smallest Weighted Averages',
                  title_x=0.5, title_font=dict(size=24))
fig.update_traces(marker=dict(line=dict(color='#000000', width=2)))
fig.update_layout({'plot_bgcolor': 'rgba(0, 0, 0, 0)'})
fig.show()

In [ ]:
# Creating a new DataFrame for unique movies with their weightedAverages and Genres
_ = data.merge(movie_vs_rating.reset_index()).drop_duplicates('title')[['title', 'genres', 'weightedAverage']]
_.head()

# Top 15 Drama Movies with the largest Weighted Averages


In [ ]:
drama_movies = _[_.genres.str.contains('Drama')].sort_values('weightedAverage', ascending=False).set_index('title')
fig = px.bar(drama_movies, x=drama_movies['weightedAverage'].nlargest(15).index,
             y=drama_movies['weightedAverage'].nlargest(15),
             text=drama_movies['weightedAverage'].nlargest(15),
             labels={
                 "x": "Movies",
                 'y': 'Weighted Rating Averages'},
             color=drama_movies['weightedAverage'].nlargest(15)
             )
fig.update_traces(textposition='outside')
fig.update_layout(title_text='Top 15 Drama Movies with the largest Weighted Averages',
                  title_x=0.5, title_font=dict(size=24))
fig.update_traces(marker=dict(line=dict(color='#000000', width=2)))
fig.update_layout({'plot_bgcolor': 'rgba(0, 0, 0, 0)'})
fig.show()

# Top 15 Drama Movies with the smallest Weighted Averages


In [ ]:
drama_movies = _[_.genres.str.contains('Drama')].sort_values('weightedAverage', ascending=False).set_index('title')
fig = px.bar(drama_movies, x=drama_movies['weightedAverage'].nsmallest(15).index,
             y=drama_movies['weightedAverage'].nsmallest(15),
             text=drama_movies['weightedAverage'].nsmallest(15),
             labels={
                 "x": "Movies",
                 'y': 'Weighted Rating Averages'},
             color=drama_movies['weightedAverage'].nsmallest(15)
             )
fig.update_traces(textposition='outside')
fig.update_layout(title_text='Top 15 Drama Movies with the smallest Weighted Averages',
                  title_x=0.5, title_font=dict(size=24))
fig.update_traces(marker=dict(line=dict(color='#000000', width=2)))
fig.update_layout({'plot_bgcolor': 'rgba(0, 0, 0, 0)'})
fig.show()

# Top 15 Action Movies with the largest Weighted Averages

In [ ]:
action_movies = _[_.genres.str.contains('Action')].sort_values('weightedAverage', ascending=False).set_index('title')
fig = px.bar(action_movies, x=action_movies['weightedAverage'].nlargest(15).index,
             y=action_movies['weightedAverage'].nlargest(15),
             text=action_movies['weightedAverage'].nlargest(15),
             labels={
                 "x": "Movies",
                 'y': 'Weighted Rating Averages'},
             color=action_movies['weightedAverage'].nlargest(15)
             )
fig.update_traces(textposition='outside')
fig.update_layout(title_text='Top 15 Action Movies with the largest Weighted Averages',
                  title_x=0.5, title_font=dict(size=24))
fig.update_traces(marker=dict(line=dict(color='#000000', width=2)))
fig.update_layout({'plot_bgcolor': 'rgba(0, 0, 0, 0)'})
fig.show()

# Top 15 Action Movies with the smallest Weighted Averages

In [ ]:
action_movies = _[_.genres.str.contains('Action')].sort_values('weightedAverage', ascending=False).set_index('title')
fig = px.bar(action_movies, x=action_movies['weightedAverage'].nsmallest(15).index,
             y=action_movies['weightedAverage'].nsmallest(15),
             text=action_movies['weightedAverage'].nsmallest(15),
             labels={
                 "x": "Movies",
                 'y': 'Weighted Rating Averages'},
             color=action_movies['weightedAverage'].nsmallest(15)
             )
fig.update_traces(textposition='outside')
fig.update_layout(title_text='Top 15 Action Movies with the smallest Weighted Averages',
                  title_x=0.5, title_font=dict(size=24))
fig.update_traces(marker=dict(line=dict(color='#000000', width=2)))
fig.update_layout({'plot_bgcolor': 'rgba(0, 0, 0, 0)'})
fig.show()

# Top 15 Sci-Fi Movies with the largest Weighted Averages

In [ ]:
sci_fi_movies = _[_.genres.str.contains('Sci-Fi')].sort_values('weightedAverage', ascending=False).set_index('title')
fig = px.bar(sci_fi_movies, x=sci_fi_movies['weightedAverage'].nlargest(15).index,
             y=sci_fi_movies['weightedAverage'].nlargest(15),
             text=sci_fi_movies['weightedAverage'].nlargest(15),
             labels={
                 "x": "Movies",
                 'y': 'Weighted Rating Averages'},
             color=sci_fi_movies['weightedAverage'].nlargest(15)
             )
fig.update_traces(textposition='outside')
fig.update_layout(title_text='Top 15 Sci-Fi Movies with the largest Weighted Averages',
                  title_x=0.5, title_font=dict(size=24))
fig.update_traces(marker=dict(line=dict(color='#000000', width=2)))
fig.update_layout({'plot_bgcolor': 'rgba(0, 0, 0, 0)'})
fig.show()

# Top 15 Sci-Fi Movies with the smallest Weighted Averages

In [ ]:
sci_fi_movies = _[_.genres.str.contains('Sci-Fi')].sort_values('weightedAverage', ascending=False).set_index('title')
fig = px.bar(sci_fi_movies, x=sci_fi_movies['weightedAverage'].nsmallest(15).index,
             y=sci_fi_movies['weightedAverage'].nsmallest(15),
             text=sci_fi_movies['weightedAverage'].nsmallest(15),
             labels={
                 "x": "Movies",
                 'y': 'Weighted Rating Averages'},
             color=sci_fi_movies['weightedAverage'].nsmallest(15)
             )
fig.update_traces(textposition='outside')
fig.update_layout(title_text='Top 15 Sci-Fi Movies with the smallest Weighted Averages',
                  title_x=0.5, title_font=dict(size=24))
fig.update_traces(marker=dict(line=dict(color='#000000', width=2)))
fig.update_layout({'plot_bgcolor': 'rgba(0, 0, 0, 0)'})
fig.show()

# Apriori Algorithm Application for Movie Recommendation Dataset

In [ ]:
# Created a (movieId: title) dictionary for all movieId's for replacing them with their names
movieId_dict = data.drop_duplicates('title')[['movieId', 'title']].set_index('movieId').to_dict()['title']

# First 5 elements of this dictionary
list(movieId_dict.items())[:5]

In [ ]:
# Creating a pivot table that has indexes as user ratings, and columns as each movie title
dataRecommendation = data.pivot(index='userId', columns='movieId', values='rating').fillna(0)

# Replacing dataRecommendation columns with the movie titles
dataRecommendation.columns = dataRecommendation.columns.map(movieId_dict)

dataRecommendation.head(10)

In [ ]:
def encode_units(k):
    if k <= 0:
        return 0
    if k >= 1:
        return 1


sets = dataRecommendation.applymap(encode_units)
sets.head()

In [ ]:
# Applying Apriori algorithm to matrix that we created before (userId-movieId) and setting min support as 0.15
frequent_itemsets = apriori(sets, min_support=0.15, use_colnames=True)

rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1.75).sort_values('lift', ascending=False)

rules = rules[['antecedents', 'consequents', 'support', 'lift', 'confidence']]
rules.head()

# Some Examples of Apriori Rules 

In [ ]:
i = random.randint(0, rules.shape[0])
print('Antecedents:', rules.iloc[i].antecedents)
print('Consequents:', rules.iloc[i].consequents)
print(f'Lift: {rules.iloc[i].lift.round(3)} & Confidence: {rules.iloc[i].confidence.round(3)}')

In [ ]:
i = random.randint(0, rules.shape[0])
print('Antecedents:', rules.iloc[i].antecedents)
print('Consequents:', rules.iloc[i].consequents)
print(f'Lift: {rules.iloc[i].lift.round(3)} & Confidence: {rules.iloc[i].confidence.round(3)} & Support: {rules.iloc[i].support.round(3)}')

In [ ]:
i = random.randint(0, rules.shape[0])
print('Antecedents:', rules.iloc[i].antecedents)
print('Consequents:', rules.iloc[i].consequents)
print(f'Lift: {rules.iloc[i].lift.round(3)} & Confidence: {rules.iloc[i].confidence.round(3)} & Support: {rules.iloc[i].support.round(3)}')

In [ ]:
i = random.randint(0, rules.shape[0])
print('Antecedents:', rules.iloc[i].antecedents)
print('Consequents:', rules.iloc[i].consequents)
print(f'Lift: {rules.iloc[i].lift.round(3)} & Confidence: {rules.iloc[i].confidence.round(3)} & Support: {rules.iloc[i].support.round(3)}')

# Let's Build Item-Based Collaborative Filtering Recommendation Engine

In [ ]:
# I will use NearestNeighbors algorithm that I learnt from the scikit-learn documentation here
knn = NearestNeighbors(n_neighbors=11, metric='cosine', algorithm='brute', n_jobs=-1)
knn.fit(dataRecommendation.values.T)

In [ ]:
# Here is our recommendations for Blade Runner, there will be 7 movie recommendations
recommendation_result = list(knn.kneighbors([dataRecommendation['Blade Runner'].values], 8))

recommendation_result  # As you can see from the results, we obtained cosine angles in the first array, and the second array gives us the dataRecommendation column order, I need to convert it to more readable form

In [ ]:
recommendations = pd.DataFrame(np.vstack((recommendation_result[1], recommendation_result[0])),
                 index=['movieId', 'Cosine_Similarity (degree)']).T
recommendations = recommendations.drop([0]).reset_index(drop=True)
recommendations  # In this step, I created a dataframe that stores the movieId (for dataRecommendation column order) and Cosine Similarity in degrees

In [ ]:
a = dataRecommendation.columns.to_frame().reset_index(drop=True).to_dict()['movieId']
recommendations.movieId = recommendations.movieId.map(a)
recommendations

# Some Movie Recommendations

In [ ]:
# Movie Recommendation as Function

def movie_recommendation(movie_name, num_of_recommendations):
    a = dataRecommendation.columns.to_frame().reset_index(drop=True).to_dict()['movieId']
    recommendation_result = list(knn.kneighbors([dataRecommendation[movie_name].values], num_of_recommendations + 1))
    recommendation_result = pd.DataFrame(np.vstack((recommendation_result[1], recommendation_result[0])),
                                         index=['movieId', 'Cosine_Similarity (degree)']).T
    recommendation_result = recommendation_result.drop([0]).reset_index(drop=True)
    recommendation_result.movieId = recommendation_result.movieId.map(a)
    return recommendation_result

In [ ]:
movie_recommendation('Amazing Spider-Man, The', 7)

In [ ]:
movie_recommendation('Mulan', 7)

In [ ]:
movie_recommendation('Final Destination', 7)

In [ ]:
movie_recommendation('Shining, The', 7)

In [ ]:
movie_recommendation('Pulp Fiction', 7)

In [ ]:
movie_recommendation("Reservoir Dogs", 7)

In [ ]:
movie_recommendation('Batman Returns', 7)

In [ ]:
movie_recommendation('Logan', 7)

In [ ]:
movie_recommendation('Matrix, The', 7)

In [ ]:
movie_recommendation('Casablanca', 7)

In [ ]:
movie_recommendation("Harry Potter and the Sorcerer's Stone", 7)

In [ ]:
movie_recommendation('Rain Man', 7)

# Thanks!